 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_returns_csv = Path("Resources/whale_returns.csv")



In [ ]:
# Count nulls
whale_returns_df = pd.read_csv(whale_returns_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
whale_returns_df.sort_index(inplace=True)



In [ ]:
# Drop nulls
whale_returns_df.isnull().sum()
whale_returns_df.dropna(inplace=True)
whale_returns_df.head()


## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_csv = Path("Resources/algo_returns.csv")


In [ ]:
# Count nulls
algo_returns_df = pd.read_csv(algo_returns_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
algo_returns_df.sort_index(inplace=True)


In [ ]:
# Drop nulls
algo_returns_df.isnull().sum()
algo_returns_df.dropna(inplace=True)
algo_returns_df.head()


## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("Resources/sp500_history.csv")


In [ ]:
# Check Data Types
sp500_history_df = pd.read_csv(sp500_history_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_history_df.sort_index(inplace=True)
sp500_history_df.head()


In [ ]:
# Fix Data Types
sp500_history_df.dtypes
sp500_history_df['Close'] = sp500_history_df['Close'].str.replace('$', '')
sp500_history_df['Close'] = sp500_history_df['Close'].astype('float')
sp500_history_df.dtypes




In [ ]:
# Calculate Daily Returns
sp500_history_df = sp500_history_df.pct_change()
sp500_history_df.head()


In [ ]:
# Drop nulls
sp500_history_df.isnull().sum()
sp500_history_df.dropna(inplace=True)
sp500_history_df.head()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sp500_history_df.rename(columns={'Close':'SP500'}, inplace=True)
sp500_history_df.head()


## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
combined_df = pd.concat([whale_returns_df, algo_returns_df, sp500_history_df], axis='columns', join='inner')
combined_df.sort_index(inplace=True)
combined_df.head()


---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
combined_df.plot(figsize=(20,10), title='Daily Returns of All Portfolios')



#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + combined_df).cumprod() - 1
cumulative_returns.head()

# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10), title='Cumulative Returns of All Portfolios')



---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
combined_df.plot.box(figsize=(20,10), title='Risk of All Portfolios')



### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
daily_std = combined_df.std()
daily_std.head()


### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate the daily standard deviation of S&P 500
sp500_std = sp500_history_df.std()
sp500_std.head()


# Determine which portfolios are riskier than the S&P 500
riskier_portfolios = daily_std > sp500_std
riskier_portfolios.head()






### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_std = daily_std * np.sqrt(252)
annualized_std.head()


---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling_std = combined_df.rolling(window=21).std()
rolling_std.head()

# Plot the rolling standard deviation
rolling_std.plot(figsize=(20,10), title='Rolling Standard Deviation of All Portfolios')



### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation = combined_df.corr()
correlation.head()

# Display de correlation matrix
import seaborn as sns
sns.heatmap(correlation, vmin=-1, vmax=1)


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
variance = combined_df['BERKSHIRE HATHAWAY INC'].var()
variance


# Calculate variance of S&P 500
sp500_variance = sp500_history_df['SP500'].var()
sp500_variance


# Computing beta
covariance = combined_df['BERKSHIRE HATHAWAY INC'].cov(combined_df['SP500'])
covariance


# Plot beta trend
rolling_covariance = combined_df['BERKSHIRE HATHAWAY INC'].rolling(window=30).cov(combined_df['SP500'])
rolling_variance = combined_df['SP500'].rolling(window=30).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.plot(figsize=(20, 10), title='Rolling 30-Day Beta of Berkshire Hathaway')





## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window
ewm = combined_df.ewm(halflife=21, adjust=True).mean()
ewm.head()


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios = (combined_df.mean() * 252) / (combined_df.std() * np.sqrt(252))
sharpe_ratios.head()



In [ ]:
# Visualize the sharpe ratios as a bar plot]
sharpe_ratios.plot.bar(title='Sharpe Ratios')



### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.
Algo 1 and S&P 500 outperform the whales portfolios. Algo 2 does not outperform the whales portfolios.


---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns

%matplotlib inline

In [ ]:
# Reading data from 1st stock
amd_df = Path("Resources/AMD.csv")
amd_df = pd.read_csv(amd_df, index_col="Date", infer_datetime_format=True, parse_dates=True)
amd_df.__str__()
str(amd_df)




In [ ]:
# Reading data from 2nd stock
intc_df = Path("Resources/INTC.csv")
intc_df = pd.read_csv(intc_df, index_col="Date", infer_datetime_format=True, parse_dates=True)
intc_df.__str__()
str(intc_df)


In [ ]:
# Reading data from 3rd stock
nvda_df = Path("Resources/NVDA.csv")
nvda_df = pd.read_csv(nvda_df, index_col="Date", infer_datetime_format=True, parse_dates=True)
nvda_df.__str__()
str(nvda_df)


In [ ]:
# Combine all stocks in a single DataFrame
combined_df = pd.concat([amd_df, intc_df, nvda_df], axis='columns', join='inner')
combined_df.sort_index(inplace=True)
combined_df.columns = ['AMD', 'INTC', 'NVDA']


In [ ]:
# Reset Date index
combined_df.reset_index(inplace=True)
combined_df.head()


In [ ]:
# Reorganize portfolio data by having a column per symbol
combined_df.drop(columns=['Date'], inplace=True)
combined_df.head()


In [ ]:
# Calculate daily returns
daily_returns = combined_df.pct_change()


# Drop NAs
daily_returns.dropna(inplace=True)


# Display sample data
daily_returns.head()


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
portfolio_returns = daily_returns.dot(weights)

# Display sample data
portfolio_returns.head()


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
combined_df = pd.concat([combined_df, portfolio_returns], axis='columns', join='inner')
combined_df.head()


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
combined_df.dropna(inplace=True)
combined_df.head()


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
std = combined_df.std() * np.sqrt(252)
std.head()


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
rolling_std = combined_df.rolling(window=21).std()
rolling_std.head()

# Plot rolling standard deviation
rolling_std.plot(figsize=(20,10), title='Rolling 21-Day Standard Deviation')



### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
correlation = combined_df.corr()
correlation.head()


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta
covariance = combined_df['AMD'].cov(combined_df['NVDA'])
variance = combined_df['NVDA'].var()
beta = covariance / variance
beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios
sharpe_ratios = (combined_df.mean() * 252) / (combined_df.std() * np.sqrt(252))
sharpe_ratios.head()


In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot.bar(title='Sharpe Ratios')



### How does your portfolio do?


My portfolio does well compared to the S&P 500. It has a higher cumulative return and a higher sharpe ratio. It also has a lower standard deviation and a lower beta. It is less risky than the S&P 500 and has a higher return.